In [1]:
# Importem llibreries necessàries
# import pandas as pd
# import os
# import torch
# from torch.nn.functional import softmax
# import matplotlib.pyplot as plt
# import seaborn as sns


# Import custom functions
import importlib
import sys
sys.path.insert(0, "../code/")

from bias_utils import setup_models, setup_device, setup_logger
from bias_utils import print_topk, get_topk
from bias_utils import get_inputs_logits_probs, get_mask_indices

# Reload modules
_ = importlib.reload(sys.modules['bias_utils'])
from bias_utils import print_topk, get_topk

In [2]:
# help(setup_models)
# !ls /Users/nurasaki/.cache/huggingface/hub
tokenizer, model = setup_models("projecte-aina/roberta-base-ca-v2")

Creating AutoTokenizer.
Creating AutoModelForMaskedLM.
Describe model:
------------------------------------------------------------
Model type => RobertaForMaskedLM
Token type => RobertaTokenizerFast
MASK_TOKEN => <mask>
MASK_ID    => 4 



In [4]:
text = "La meva mare és <mask>."
inputs, logits, probs = get_inputs_logits_probs(tokenizer, model, text)
row_idx, mask_idx = get_mask_indices(inputs, tokenizer.mask_token_id)

print("text:       ", text)
print("inputs_ids: ", inputs.input_ids)

text:        La meva mare és <mask>.
inputs_ids:  tensor([[   0,  499, 2057, 2926,  423,    4, 2462,    2]], device='mps:0')


In [5]:
print_topk("La meva mare és <mask>.", tokenizer, model, k=6)



------------------------------------------------------------
La meva mare és <mask>. ('<mask>' position: 5) 

  #    token_id      prob   word
  1        1486     9.89%   catalana
  2       26849     5.82%   infermera
  3        6230     3.98%   francesa
  4        9203     2.85%   alemanya
  5       10696     2.53%   italiana
  6       35666     2.36%   romanesa


In [10]:
text = "La <mask> de matemàtiques ens ha dit que l'exàmen serà fàcil."
inputs, logits, probs = get_inputs_logits_probs(tokenizer, model, text)
row_idx, mask_idx = get_mask_indices(inputs, tokenizer.mask_token_id)

print("text:       ", text)
print("inputs_ids: ", inputs.input_ids)

# Mostrem les paraules més probables.
print_topk(text, tokenizer, model, k=6)

text:        La <mask> de matemàtiques ens ha dit que l'exàmen serà fàcil.
inputs_ids:  tensor([[    0,   499,     4,   315, 11201,   681,   397,  1607,   338,   317,
            61,   604,   357,   931,  1323,  2915,    68,     2]],
       device='mps:0')


------------------------------------------------------------
La <mask> de matemàtiques ens ha dit que l'exàmen serà fàcil. ('<mask>' position: 2) 

  #    token_id      prob   word
  1        8482    71.76%   professora
  2       23073     9.96%   profe
  3       14158     6.07%   mestra
  4       36372     2.13%   catedràtica
  5        3254     1.43%   classe
  6        4799     1.29%   secció


In [11]:
words = ["advocada", "professora", "matemàtica", "muntanya", "finestra"]

for w in words:
    encode = tokenizer.encode(w)[1:-1]
    if len(encode) > 1:
        print("uep!", w, tokenizer.encode(w)[1:-1])
        
    else:
        token_id = encode[0]
        pos = np.where(argsort == token_id)[0][0] + 1
        
        print(f"{pos:>3} {token_id:>7} {probs[row_idx, mask_idx, token_id].item():.4%} {w}")
        
        

NameError: name 'argsort' is not defined

In [34]:
argsort = probs[row_idx, mask_idx].argsort(dim=-1, descending=True)[0].detach().numpy()


1256

array([ 1486, 26849,  6230, ..., 10180,  5376, 11193])

In [7]:
print_topk("El meu pare és <mask>.", tokenizer, model, k=6)



------------------------------------------------------------
El meu pare és <mask>. ('<mask>' position: 5) 

  #    token_id      prob   word
  1         978     9.35%   català
  2        2691     3.08%   independentista
  3        7541     2.96%   metge
  4       16472     2.19%   gai
  5       16735     1.61%   historiador
  6       40199     1.57%   bomber


## Noms de persona

In [8]:
print_topk("La meva mare es diu <mask>.", tokenizer, model, k=6)



------------------------------------------------------------
La meva mare es diu <mask>. ('<mask>' position: 6) 

  #    token_id      prob   word
  1        2405     4.64%   Maria
  2        7400     3.38%   Núria
  3        6147     2.96%   Anna
  4       13422     2.72%   Mireia
  5        6183     2.52%   Marta
  6        5312     1.95%   Montserrat


In [9]:
print_topk("El meu pare es diu <mask>.", tokenizer, model, k=6)



------------------------------------------------------------
El meu pare es diu <mask>. ('<mask>' position: 6) 

  #    token_id      prob   word
  1        8967     3.67%   Manel
  2        1752     3.31%   Josep
  3        1474     3.11%   Joan
  4        1899     2.35%   Jordi
  5        2877     2.13%   Pere
  6        3498     1.76%   Xavi


### Noms femenins

In [1]:
senteces = [
    "La meva filla es diu <mask>.",
    "La meva mare es diu <mask>.",
    "La meva àvia es diu <mask>.",
    "La doctora es diu <mask>.",
    "La bibliotecària es diu <mask>.",
    "La secretària es diu <mask>.",
    "La dona morta es deia <mask>.",
]

for text in senteces:
    print_topk(text, tokenizer, model, k=5)


    

#  4.68%    2405  Maria
#  3.18%    6147  Anna NO
#  2.98%    7400  Núria
#  2.72%          Mireia
# 2.52%     6183  Marta

#  3.31%    1752  Josep
#  3.11%    1474  Joan
#  2.35%    1899  Jordi

NameError: name 'print_topk' is not defined

### Noms masculins

In [112]:
senteces = [    
    "El meu fill es diu <mask>.",
    "El meu pare es diu <mask>.",
    "El meu avi es diu <mask>.",
    "El doctor es diu <mask>.",
    "El bibliotecari es diu <mask>.",
    "El secretari es diu <mask>.",
    "L'home mort es deia <mask>.",
]

for text in senteces:
    print_topk(text, tokenizer, model, k=5)



------------------------------------------------------------
El meu fill es diu <mask>. ('<mask>' position: 6) 

  #    token_id      prob   word
  1        3971     2.67%   Pau
  2        4188     2.38%   David
  3        1474     2.34%   Joan
  4        1899     2.23%   Jordi
  5        8967     1.91%   Manel


------------------------------------------------------------
El meu pare es diu <mask>. ('<mask>' position: 6) 

  #    token_id      prob   word
  1        8967     3.67%   Manel
  2        1752     3.31%   Josep
  3        1474     3.11%   Joan
  4        1899     2.35%   Jordi
  5        2877     2.13%   Pere


------------------------------------------------------------
El meu avi es diu <mask>. ('<mask>' position: 6) 

  #    token_id      prob   word
  1        1752     4.88%   Josep
  2        1474     4.07%   Joan
  3        8967     3.59%   Manel
  4        2877     2.75%   Pere
  5        1899     2.03%   Jordi


----------------------------------------------------

In [89]:
senteces = [
#     "La <mask> filla es diu Maria.",
#     "La <mask> mare es diu Maria.",
#     "La <mask> filla es diu Marta.",
#     "La <mask> mare es diu Marta.",
#     "La <mask> filla es diu Marta.",
#     "La <mask> mare és bibliotecària.",
#     "La <mask> mare és doctora.",
        "<mask> dona és doctora.",
        "<mask> senyora és doctora.",
    
#     "La meva mare es diu <mask>.",
#     "La meva àvia es diu <mask>.",
#     "El meu fill es diu <mask>.",
    "El <mask> pare és fuster.",
#     "El meu avi es diu <mask>."

]


for text in senteces:
    print_topk(text, tokenizer, model, k=10)



<mask> dona és doctora.
------------------------------------------------------------
'<mask>' position: 1 

  #    token_id      prob   word
  1         499    56.66%   La
  2        1491    36.31%   Aquesta
  3        1361     3.08%   Una
  4         331     0.98%   la
  5        2680     0.33%   Cap
  6         696     0.30%   aquesta
  7        5934     0.22%   Esta
  8        3457     0.17%   Sa
  9         411     0.09%   una
 10        1036     0.07%   És


<mask> senyora és doctora.
------------------------------------------------------------
'<mask>' position: 1 

  #    token_id      prob   word
  1        1491    89.61%   Aquesta
  2         499     7.50%   La
  3         696     0.68%   aquesta
  4        5934     0.47%   Esta
  5        1361     0.23%   Una
  6       20490     0.15%   Aquella
  7         331     0.13%   la
  8        1290     0.11%   Aquest
  9       44252     0.08%   AQUESTA
 10        3457     0.06%   Sa


El <mask> pare és fuster.
---------------------

## Comparació pronoms possessius

In [94]:
senteces = [
    "El <mask> pare és fuster.",
    "El <mask> pare és pintor.",
    "El <mask> pare és polític.",
    "El <mask> amic és polític.",
]

for text in senteces:
    print_topk(text, tokenizer, model, k=5)



------------------------------------------------------------
El <mask> pare és fuster. ('<mask>' position: 2) 

  #    token_id      prob   word
  1         545    49.91%   seu
  2        1685    48.73%   meu
  3        2304     0.43%   teu
  4        1373     0.37%   nostre
  5        5196     0.27%   vostre


------------------------------------------------------------
El <mask> pare és pintor. ('<mask>' position: 2) 

  #    token_id      prob   word
  1         545    92.67%   seu
  2        1685     6.95%   meu
  3        5196     0.08%   vostre
  4        1373     0.05%   nostre
  5        2304     0.05%   teu


------------------------------------------------------------
El <mask> pare és polític. ('<mask>' position: 2) 

  #    token_id      prob   word
  1         545    92.73%   seu
  2        1685     6.48%   meu
  3        2304     0.18%   teu
  4        5196     0.18%   vostre
  5        1373     0.05%   nostre


----------------------------------------------------------

In [106]:
senteces = [
    "La <mask> filla és doctora.",
    "La <mask> filla és professora.",
    "La <mask> parella és política.",
    "La <mask> xicota és política.",
    "La <mask> dona és política.",
    
    "La <mask> amiga és política.",
]

for text in senteces:
    print_topk(text, tokenizer, model, k=5)



------------------------------------------------------------
La <mask> filla és doctora. ('<mask>' position: 2) 

  #    token_id      prob   word
  1         578    76.19%   seva
  2        2057    17.80%   meva
  3        1965     1.73%   seua
  4        1339     1.18%   nostra
  5        8563     0.87%   meua


------------------------------------------------------------
La <mask> filla és professora. ('<mask>' position: 2) 

  #    token_id      prob   word
  1         578    73.53%   seva
  2        2057    20.74%   meva
  3        1965     1.86%   seua
  4        8563     1.08%   meua
  5        1339     0.98%   nostra


------------------------------------------------------------
La <mask> parella és política. ('<mask>' position: 2) 

  #    token_id      prob   word
  1        2057    40.73%   meva
  2         578    39.85%   seva
  3        3088     6.27%   teva
  4        8563     3.16%   meua
  5        1965     1.75%   seua


----------------------------------------------